<a href="https://colab.research.google.com/github/lanukahjo/Image-Splicing-Localisation/blob/main/Image_Splicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Splicing Localisation
#### This notebook is the main flow of execution

In [37]:
from skimage import data
from skimage import filters
from skimage import io
from skimage.feature import greycomatrix
import numpy as np
from skimage import color
import os
import math

Reading the file from the given location
Manually for now....

In [38]:
# PATH_TO_AUTHENTIC_IMG = os.getcwd() + '/ImSpliceDataset/Au-S/AU_S_026.bmp'
# PATH_TO_SPLICED_IMG = os.getcwd() + '/ImSpliceDataset/Sp-S/SP_S_026.bmp';
PATH_TO_AUTHENTIC_IMG = '/content/AU_S_013.bmp'
PATH_TO_SPLICED_IMG = '/content/SP_S_013.bmp';
THRESHOLD = 0.3;
BLOCK_SIZE = 4;
STEP_SIZE = 2;

###Assumption
- Images are already in grayscale format (0-255)


In [39]:
authentic_img = io.imread(PATH_TO_AUTHENTIC_IMG);
spliced_img = io.imread(PATH_TO_SPLICED_IMG); 

In [40]:
output_img_euclidean_dist = spliced_img;
output_img_hellinger_dist = spliced_img;

authentic_img_grey_reduced = (authentic_img/32).astype(int);
spliced_img_grey_reduced = (spliced_img/32).astype(int);

img_height = authentic_img.shape[0]
img_width = authentic_img.shape[1]

In [41]:
# Get Hellinger Distance between two numpy ndarrays x and y
def getHellingerDistance(x, y):
  dot_product = np.multiply(x,y);
  squared_matrix = x - 2*np.sqrt(dot_product) + y;
  hellinger_distance = math.sqrt(np.sum(squared_matrix)/2.0); 
  return hellinger_distance;

In [42]:
# Get euclidean distance between two numpy ndarrays x and y
def getEuclideanDistance(x, y):
  euclidean_distance = math.sqrt(np.sum(np.square(x-y)))
  return euclidean_distance; 

###Terminology
**Distance Matrix**: The matrix of blocks obtained by splitting the image into set of overlapping blocks of size BLOCK_SIZE with a step size of STEP_SIZE. 
**dist_mat_height** and **dist_mat_width** are the height and width of distance matrix respectively

In [43]:
# We shall store the euclidean and hellinger distances because we need to normalise the distances too
dist_mat_height = (int)((img_height - BLOCK_SIZE)/STEP_SIZE + 1);
dist_mat_width = (int)((img_width - BLOCK_SIZE)/STEP_SIZE + 1);
euc_dist_mat = np.zeros((dist_mat_height,dist_mat_width),dtype=float)
hell_dist_mat = np.zeros((dist_mat_height,dist_mat_width),dtype=float)

In [44]:
# row and col are used to index into the distance matrix
row = 0;
col = 0;
for i in range(BLOCK_SIZE-1,img_height,STEP_SIZE):
  col = 0;
  for j in range(BLOCK_SIZE-1,img_width,STEP_SIZE):
    authentic_block = authentic_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    spliced_block = spliced_img_grey_reduced[i+1-BLOCK_SIZE:i+1, j+1-BLOCK_SIZE:j+1]
    
    glcm_matrix_auth = greycomatrix(authentic_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
    glcm_matrix_spl = greycomatrix(spliced_block,distances=[1],angles=[0],levels = 8,normed=True)[:,:,0,0]
  
    hellinger_distance = getHellingerDistance(glcm_matrix_auth, glcm_matrix_spl);
    euclidean_distance = getEuclideanDistance(glcm_matrix_auth, glcm_matrix_spl);
    hell_dist_mat[row,col] = hellinger_distance
    euc_dist_mat[row,col] = euclidean_distance

    col = col + 1
  row = row + 1

In [45]:
# Normalize the distance matrix to have values between 0~1
euc_dist_mat = euc_dist_mat / np.max(euc_dist_mat)
hell_dist_mat = hell_dist_mat / np.max(hell_dist_mat)

In [46]:
for i in range(0,dist_mat_height):
    for j in range(0,dist_mat_width):
        if (euc_dist_mat[i,j] > THRESHOLD):
            output_img_euclidean_dist[i*STEP_SIZE:i*STEP_SIZE+BLOCK_SIZE, j*STEP_SIZE:j*STEP_SIZE+BLOCK_SIZE] = 255*np.ones((BLOCK_SIZE,BLOCK_SIZE))
        if (hell_dist_mat[i,j] > THRESHOLD):
            output_img_hellinger_dist[i*STEP_SIZE:i*STEP_SIZE+BLOCK_SIZE, j*STEP_SIZE:j*STEP_SIZE+BLOCK_SIZE] = 255*np.ones((BLOCK_SIZE,BLOCK_SIZE))


In [47]:
# Find the image name from the file path bounded by left and right borders
# left_border = PATH_TO_SPLICED_IMG.rindex("/")
# right_border = PATH_TO_SPLICED_IMG.rindex(".")
# base_file_name = PATH_TO_SPLICED_IMG[left_border:right_border]

# Save the output image
# io.imsave('output/Sp-S' + base_file_name + '_euclidean.bmp', output_img_euclidean_dist)
# io.imsave('output/Sp-S' + base_file_name + '_hellinger.bmp', output_img_hellinger_dist
io.imsave('output_euclidean.bmp', output_img_euclidean_dist);
io.imsave('output_hellinger.bmp', output_img_hellinger_dist);